In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [2]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,2], inplace = True)
labelsTrain.columns = [ "eye_color" , "img_name"]
labelsTrain = labelsTrain.astype({'eye_color': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,2], inplace = True)
labelsTest.columns = [ "eye_color" , "img_name"]
labelsTest = labelsTest.astype({'eye_color': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [ ]:
#230:270,180:320

In [531]:
#Load training set################################
imagesTrain = np.zeros((10000, 50 , 50, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[230:270,180:320] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = image
    
imagesTrain = imagesTrain.reshape(10000,7500)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 50 , 50, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[230:270,180:320]
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = image
    
    
imagesTest = imagesTest.reshape(2500,7500)   
imagesTest = pd.DataFrame(imagesTest) 

In [532]:
x_train=np.zeros((10000,7502))
for i in range(0,10000):
    x_train[i]=np.convolve(imagesTrain.iloc[i,:],[0.066,-5.599,5.566])
    
x_test=np.zeros((2500,7502))
for i in range(0,2500):
    x_test[i]=np.convolve(imagesTest.iloc[i,:],[0.066,-5.599,5.566])

x_train = pd.DataFrame(x_train) 
x_test = pd.DataFrame(x_test)
x_train = pd.concat([labelsTrain,x_train],axis=1, join='inner')
#x_test = pd.concat([labelsTest,x_test],axis=1, join='inner')

In [533]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,7492,7493,7494,7495,7496,7497,7498,7499,7500,7501
0,12.012,-1005.488,-118.217,-257.785,850.487,-93.885,-209.770,753.093,-86.119,-151.074,...,1419.330,0.000,12.012,-1005.488,-118.217,-252.307,391.578,-118.217,-264.319,1397.066
1,11.682,-977.823,-119.108,-200.255,1115.807,-115.357,-442.288,597.971,-115.423,-453.420,...,19.019,-118.910,-162.360,302.885,-118.910,-162.360,302.885,-118.910,-175.098,1369.236
2,8.448,-704.726,-285.923,-265.694,986.282,-410.157,-263.846,680.185,-410.124,-263.351,...,25.146,-2.684,-5.731,731.269,-291.555,-260.744,564.322,-291.522,-269.093,1263.482
3,6.270,-521.807,-316.327,-120.901,363.319,-282.535,-122.012,18.854,-130.339,-156.530,...,14.014,2.849,-0.363,745.228,-72.116,-103.730,-198.055,-118.910,-175.098,1369.236
4,5.544,-462.660,-170.720,-300.630,483.560,-170.720,-300.630,483.560,-170.720,-300.630,...,-217.558,-181.654,-306.361,511.390,-170.720,-300.630,483.560,-170.720,-306.174,946.220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,1.188,-99.462,-10.010,-38.929,72.820,0.924,-38.929,55.957,6.292,-31.548,...,709.071,-285.956,-255.244,575.454,-291.522,-260.777,569.888,-291.522,-269.093,1263.482
2496,10.164,-852.082,5.082,5.082,5.082,5.082,5.082,5.082,5.016,10.945,...,1231.538,-26.741,28.908,-921.305,-118.646,-157.091,352.979,-118.910,-175.098,1369.236
2497,4.224,-349.426,-387.629,-264.110,724.713,-410.124,-263.879,685.751,-410.124,-263.879,...,716.837,-12.474,4.873,420.024,-64.086,-34.958,-593.604,-118.217,-264.319,1397.066
2498,13.794,-1155.737,-46.321,-172.601,241.329,-46.321,-173.129,285.659,-52.184,-176.605,...,1419.330,0.000,13.794,-1155.737,-46.321,-172.601,241.329,-46.321,-186.395,1397.066


In [540]:
bag1

,eye_color,img_name,0,1,2,3,4,5,6,7,...,7492,7493,7494,7495,7496,7497,7498,7499,7500,7501
7282,3,7282.png,6.930,-579.513,-116.413,-153.505,237.897,-29.139,-25.784,178.651,...,-249.766,-252.791,-345.719,629.497,-247.489,-334.389,596.101,-247.489,-340.527,1107.634
6410,4,6410.png,12.012,-1005.488,-118.217,-252.307,391.578,-118.217,-252.307,391.578,...,1296.911,0.033,-0.033,5.566,0.000,12.210,-1022.021,-123.651,-264.451,1419.330
3777,4,3777.png,13.794,-1155.737,-46.321,-172.601,241.329,-46.321,-172.601,241.329,...,89.518,-4.147,-83.622,89.518,-4.147,-83.622,89.518,-4.147,-84.414,155.848
4667,3,4667.png,6.402,-532.939,-310.530,-138.787,474.441,-310.728,-127.523,452.177,...,50.127,40.282,-88.858,50.127,40.282,-88.858,50.127,40.282,-89.848,133.584
2900,2,2900.png,1.254,-104.071,-86.119,-151.074,241.021,-86.119,-151.074,241.021,...,8.415,8.415,-6.105,1226.533,-68.332,-205.579,208.384,-68.035,-197.978,534.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2417,3,2417.png,12.078,-1011.021,-118.118,-259.853,561.649,4.983,11.506,-71.786,...,694.573,-1.276,5.269,391.963,-24.926,-51.755,-576.906,-118.217,-264.319,1397.066
1340,4,1340.png,0.924,-77.396,-4.609,-37.840,-32.076,-31.020,-104.335,30.272,...,1181.048,-26.015,-76.109,-36.520,-68.035,-194.810,269.610,-68.035,-197.978,534.336
4502,1,4502.png,6.138,-511.533,-247.489,-334.389,596.101,-247.489,-334.851,634.634,...,679.162,-99.055,-138.556,451.682,-258.126,-357.016,657.327,-247.489,-340.527,1107.634
936,2,936.png,7.854,-658.823,36.201,82.511,-107.789,36.201,82.511,-107.789,...,1419.594,-22.396,29.524,-605.264,-274.857,-311.597,602.558,-274.857,-318.857,1207.822


In [556]:
bag1['eye_color']

7473    3
3592    2
5829    3
8927    0
9382    0
       ..
1903    2
410     0
3602    4
4449    1
7467    4
Name: eye_color, Length: 1000, dtype: int32

In [ ]:
for i in range(0,6):
    accuracy = []
    bag1 = x_train.sample(1500,replace = True)
    r=range(41,161,20) #peaks at 950/1000
    for i in r: 
        model = KNeighborsClassifier(n_neighbors=i, weights = 'uniform' , n_jobs=-1)
        model.fit(bag1.iloc[:,2:],bag1['eye_color'])
        accuracy.append(model.score(x_test,labelsTest['eye_color']))
    plt.plot(r,accuracy)




In [465]:
# model = tree.DecisionTreeClassifier(max_depth=4,criterion = 'gini')
# model.fit(x_train,labelsTrain['eye_color'])
# model.score(x_test,labelsTest['eye_color'])

In [499]:
predictions = model.predict(x_test) # misclassifying all the glasses
count=0
index=[]
for d, prediction, label in zip(x_test, predictions, labelsTest['eye_color']):

    if prediction != label:
        print(count,d, 'has been classified as ', prediction, 'and should be ', label)
        index.append(count)
    count+=1

5 [   7.854 -658.823   36.201 ...  -15.807   -6.424  144.716] has been classified as  3 and should be  1
6 [   9.768 -818.092  -60.786 ...  -59.906  -56.65   166.98 ] has been classified as  3 and should be  2
13 [   5.544 -462.66  -170.72  ...   -3.949 -117.81   155.848] has been classified as  3 and should be  0
22 [   4.95  -414.051  -71.885 ...  -59.906  -56.65   166.98 ] has been classified as  3 and should be  1
23 [   12.738 -1066.351  -118.91  ...   -14.696  -892.881  1330.274] has been classified as  3 and should be  1
27 [   13.794 -1155.737   -46.321 ...    73.414   -45.32    133.584] has been classified as  3 and should be  1
31 [   5.544 -462.66  -170.72  ...   -4.279  -51.084  166.98 ] has been classified as  3 and should be  4
44 [   5.874 -489.401 -249.667 ...   73.414  -45.32   133.584] has been classified as  3 and should be  1
49 [ 4.95000e+00 -4.14051e+02 -7.18850e+01 ...  5.04900e+01 -3.96000e-01
  6.67920e+01] has been classified as  3 and should be  1
50 [   6.13